In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [4]:
import re
import datasets
import builtins
import pandas as pd
# from vllm import LLM, SamplingParams, AsyncLLMEngine, AsyncEngineArgs
import os
from typing import List
import sys
sys.path.append("/home/franlucc/projects/codetrace")
DIR="/home/franlucc/projects/codetrace"
EXPDIR="/mnt/ssd/franlucc"
from codetrace.utils import load
from codetrace.type_inf_exp.py_mutator import IMPORT_STATEMENT_QUERY, get_captures
import typing
from codetrace.parsing_utils import placeholder_to_std_fmt, DEEPSEEK_FIM

# Codellama completions

In [3]:

ds = datasets.load_from_disk(f"{DIR}/experiments/codellama_7b/py_completions")
ds_sc1_ = datasets.load_dataset("nuprl-staging/py_typeinf_fim", split="train")
ds, ds_sc1_

(Dataset({
     features: ['key', 'prefix', 'suffix', 'middle', 'correct', 'model', 'fim_type', 'fim_program', 'hexsha', 'generated_text'],
     num_rows: 125507
 }),
 Dataset({
     features: ['key', 'prefix', 'suffix', 'middle', 'correct', 'model', 'fim_type', 'fim_program', 'hexsha'],
     num_rows: 265879
 }))

In [4]:
hexshas = set(ds["fim_program"])
ds_sc1 = ds_sc1_.filter(lambda x: x["fim_program"] in hexshas, num_proc=30)

In [5]:
df_sc1 = ds_sc1.to_pandas()
df_sc1_ = ds_sc1_.to_pandas()
df = ds.to_pandas()
print(df_sc1["correct"].value_counts(), df_sc1["correct"].mean())
print(df_sc1_["correct"].value_counts(), df_sc1_["correct"].mean())
print(df["correct"].value_counts(), df["correct"].mean())

correct
True     118953
False      6554
Name: count, dtype: int64 0.9477798051104719
correct
True     134511
False    131368
Name: count, dtype: int64 0.5059105833856754
correct
False    113228
True      12279
Name: count, dtype: int64 0.09783518050786012


In [6]:
def get_pattern(fim_type:str) -> str:
    raw_pattern = r'from\s+.+?\s+import\s+({fim_type})\n|import\s*.*?\s+?({fim_type})\n'
    return raw_pattern.format(fim_type=fim_type)

def is_imported(fim_type:str, fim_program:str) -> bool:
    
    matches = re.search(get_pattern(fim_type), fim_program)
    if not matches:
        return False
    # print([matches.group(i) for i in range(1, len(matches.groups()) + 1)])
    return any(matches.group(i) == fim_type for i in range(1, len(matches.groups()) + 1))

def is_imported_ts(fim_type:str, fim_program:str)->bool:
    prog = fim_program.replace("<FILL>","_placeholder_")
    captures = get_captures(prog, IMPORT_STATEMENT_QUERY, "py", "import_statement")
    for cap in captures:
        imported_statements = cap.text.decode("utf-8")
        match = re.search(r'\b({fim_type})\b'.format(fim_type=fim_type), imported_statements)
        if match != None:
            return True
    return False
    
def is_builtin(fim_type:str)->bool:
    return any([fim_type==str(i) for i in dir(builtins)+dir(typing)])

prog = '''from  oop import strop
from oop.dilup import strop
import Strofhsugf
import igfhi from str

from j import (
    strop
)

from j.model import Integration, stroop

'''
is_imported_ts("s", prog)

False

In [7]:
ds = ds.map(lambda x: {**x, 
                       "is_imported_fim" : is_imported(x["fim_type"], x["fim_program"]),
                       "is_builtin": any([x["fim_type"]==str(i) for i in dir(builtins)]),
                       }, num_proc=30)

Map (num_proc=30): 100%|██████████| 125507/125507 [00:02<00:00, 45311.28 examples/s] 


In [8]:
df = ds.to_pandas()
df["imported_fim_or_builtin"] = df["is_builtin"] | df["is_imported_fim"]
df["imported_fim_and_builtin"] = df["is_builtin"] & df["is_imported_fim"]
df_correct = df[df["correct"]]
df_incorrect = df[~df["correct"]]
print(df["is_imported_fim"].value_counts())
print((df_correct["is_imported_fim"]).value_counts())
print(df["correct"].mean())

is_imported_fim
False    106442
True      19065
Name: count, dtype: int64
is_imported_fim
False    9998
True     2281
Name: count, dtype: int64
0.09783518050786012


In [9]:
def ignore_types(d, types):
    for t in types:
        d = d[d["fim_type"] != str(t)]
    return d.reset_index()
 
df_corr_imported = df_correct[df_correct["is_imported_fim"]].reset_index()
df_incorr_imported = df_incorrect[df_incorrect["is_imported_fim"]].reset_index()
# df_ic = ignore_types(df_, dir(builtins))
# k=2
# print(df_ic["fim_program"][k], df_ic["fim_type"][k], df_ic["is_imported_fim"][k])

# with pd.option_context('display.max_rows', None):
#     print(df_ic["fim_type"].value_counts())

print(df_correct["is_imported_fim"].mean())
print(df_correct["is_builtin"].mean())

print(df_correct["is_imported_fim"].value_counts())
print(df_correct["is_builtin"].value_counts())
print(df_correct["imported_fim_or_builtin"].value_counts())
print(df_correct["imported_fim_and_builtin"].value_counts())

print(df_correct["is_imported_fim"].sum() / df_correct["imported_fim_or_builtin"].sum())
print(df_correct["is_builtin"].sum() / df_correct["imported_fim_or_builtin"].sum())
print(df_correct["imported_fim_or_builtin"].mean())

print(df_correct[df_correct["imported_fim_and_builtin"]]["fim_type"].value_counts())
item = df_correct[df_correct["imported_fim_and_builtin"]].reset_index()
# print(item["fim_program"][0], item["fim_type"][0])

0.18576431305480903
0.7217200097727828
is_imported_fim
False    9998
True     2281
Name: count, dtype: int64
is_builtin
True     8862
False    3417
Name: count, dtype: int64
imported_fim_or_builtin
True     11138
False     1141
Name: count, dtype: int64
imported_fim_and_builtin
False    12274
True         5
Name: count, dtype: int64
0.20479439755790985
0.7956545160711079
0.9070771235442625
fim_type
bool    3
int     2
Name: count, dtype: int64


In [10]:
muts = pd.read_csv(f"{DIR}/experiments/codellama_7b/nov7_test_all_v2_incorrect_log.csv")

In [11]:
k=67
muts["generated_text"][k],muts["mutated_generated_text"][k],muts["fim_type"][k]

(nan, '\n', 'float')

In [12]:
muts["mutated_generated_text"].value_counts()

mutated_generated_text
\n    1353
""      21
Name: count, dtype: int64

In [13]:
df_incorrect["generated_text"].value_counts()

generated_text
            109052
""            2164
#              314
"""            229
~~~~~~~~       168
             ...  
any              1
subset           1
tp               1
sound            1
Se               1
Name: count, Length: 211, dtype: int64

In [14]:
df["correct"].mean(), df["correct"].count()

(0.09783518050786012, 125507)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_name="/home/arjun/models/CodeLlama-7b-hf"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
).to("cuda:0")
tokenizer = AutoTokenizer.from_pretrained(model_name)

/mnt/ssd/franlucc/envs/interp/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
import datasets

def decode(outputs):
    logits = torch.softmax(outputs.logits, dim=-1)
    return torch.argmax(logits)

def generate(model, prompt, tokenizer):
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    print(input_ids)
    outputs = decode(model.forward(input_ids))
    return tokenizer.decode(outputs)

def codellama_fim_generate(model, prefix, suffix, tokenizer):
    prefix_token_id = torch.tensor([[32007]])
    suffix_token_id = torch.tensor([[32008]])
    mid_token_id = torch.tensor([[32009]])
    eos_token_id = torch.tensor([[32010]])
    prefix = tokenizer(prefix, return_tensors="pt", add_special_tokens=False)["input_ids"]
    suffix = tokenizer(suffix, return_tensors="pt", add_special_tokens=False)["input_ids"]
    input_ids = torch.concatenate([prefix_token_id, prefix, suffix_token_id, suffix,mid_token_id, eos_token_id], dim=-1)
    print(input_ids)
    outputs = decode(model.forward(input_ids.to(model.device)))
    return tokenizer.decode(outputs)

ds = datasets.load_from_disk(f"{EXPDIR}/experiments/codellama_7b/py_completions")
df = ds.to_pandas()
ds_incorrect = df[~df["correct"]].to_dict(orient="records")
ds_incorrect[0]

{'key': 'archives/sara0871_desktop.zip/homeassistant/components/bmw_connected_drive/__init__.py',
 'prefix': '"""\nReads vehicle status from BMW connected drive portal.\n\nFor more details about this platform, please refer to the documentation at\nhttps://home-assistant.io/components/bmw_connected_drive/\n"""\nimport datetime\nimport logging\n\nimport voluptuous as vol\n\nfrom homeassistant.const import CONF_USERNAME, CONF_PASSWORD\nfrom homeassistant.helpers import discovery\nfrom homeassistant.helpers.event import track_utc_time_change\nimport homeassistant.helpers.config_validation as cv\n\nREQUIREMENTS = [\'bimmer_connected==0.5.1\']\n\n_LOGGER = logging.getLogger(__name__)\n\nDOMAIN = \'bmw_connected_drive\'\nCONF_REGION = \'region\'\nATTR_VIN = \'vin\'\n\nACCOUNT_SCHEMA = vol.Schema({\n    vol.Required(CONF_USERNAME): cv.string,\n    vol.Required(CONF_PASSWORD): cv.string,\n    vol.Required(CONF_REGION): vol.Any(\'north_america\', \'china\',\n                                     

In [6]:
from codetrace.parsing_utils import get_model_fim, placeholder_to_std_fmt
fim_obj = get_model_fim(model_name)
item = ds_incorrect[-10]
prompt = placeholder_to_std_fmt(item["fim_program"], fim_obj)
generated = generate(model, prompt, tokenizer)
# print(item["fim_program"])
generated, item["fim_type"], item["generated_text"]

tensor([[    1, 32007, 29937,  ..., 29897,    13, 32009]], device='cuda:0')


('', 'str', '')

In [7]:
generated = codellama_fim_generate(model, item["prefix"], item["suffix"], tokenizer)
generated, item["fim_type"], item["generated_text"]

tensor([[32007,   396, 14187,  ...,    13, 32009, 32010]])


('', 'str', '')

# Deepseek completions

In [15]:
py_data = datasets.load_dataset("nuprl-staging/py_typeinf_fim")
py_data

DatasetDict({
    train: Dataset({
        features: ['key', 'prefix', 'suffix', 'middle', 'correct', 'model', 'fim_type', 'fim_program', 'hexsha'],
        num_rows: 265879
    })
})

In [16]:
def _filter_not_imported_or_builtin(x):
    try:
        res = (not is_builtin(x["fim_type"]) and not is_imported_ts(x["fim_type"], x["fim_program"]))
    except:
        res=False
    return res

ds_annotated = py_data.filter(_filter_not_imported_or_builtin,num_proc=30)

In [17]:
ds_annotated

DatasetDict({
    train: Dataset({
        features: ['key', 'prefix', 'suffix', 'middle', 'correct', 'model', 'fim_type', 'fim_program', 'hexsha'],
        num_rows: 77190
    })
})

In [18]:
print(ds_annotated["train"][30]["fim_type"])
ds_annotated.push_to_hub("franlucc/py_typeinf_fim_user-defined-types", private=True)

BootableService


Uploading the dataset shards: 100%|██████████| 2/2 [00:05<00:00,  2.94s/it]
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/franlucc/py_typeinf_fim_user-defined-types/commit/7bf9fdd87d4b41f0bbf22a552985cfac7ef08947', commit_message='Upload dataset', commit_description='', oid='7bf9fdd87d4b41f0bbf22a552985cfac7ef08947', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
model_name = "/mnt/ssd/franlucc/models/deepseek-coder-6.7b-base"
# model = LLM(model_name, device="cuda")
model = AsyncLLMEngine.from_engine_args(
    AsyncEngineArgs(model_name)
)

Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}


WARNING 11-08 18:00:52 arg_utils.py:766] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-08 18:00:52 config.py:820] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-08 18:00:52 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/mnt/ssd/franlucc/models/deepseek-coder-6.7b-base', speculative_config=None, tokenizer='/mnt/ssd/franlucc/models/deepseek-coder-6.7b-base', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=65536, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=aut

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.98it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.08s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.01it/s]



INFO 11-08 18:01:10 model_runner.py:732] Loading model weights took 12.5708 GB
INFO 11-08 18:01:11 gpu_executor.py:102] # GPU blocks: 7446, # CPU blocks: 512
INFO 11-08 18:01:12 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-08 18:01:12 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-08 18:01:22 model_runner.py:1225] Graph capturing finished in 10 secs.


In [20]:
def generate(llm: LLM, prompts:List[str])->List[str]:
    params = SamplingParams(max_tokens=1, temperature=0, n=1)
    outputs = llm.generate(prompts, sampling_params=params, request_id=0)
    res =[]
    for req_output in outputs:
        for output in req_output.outputs:
            res.append(output.text)
    return res

async def async_generate(llm: LLM, prompts:List[str], request_id:int)->List[str]:
    params = SamplingParams(max_tokens=1, temperature=0, n=1)
    outputs = llm.generate(prompts, sampling_params=params, request_id=request_id)
    res =[]
    async for req_output in outputs:
        for output in req_output.outputs:
            if output.text != "" and output.text != None:
                res.append(output.text)
    return res

In [21]:
prompt = '''
def factorial(n: <FILL>):
    pass
'''.strip()

await async_generate(model, placeholder_to_std_fmt(prompt, DEEPSEEK_FIM))

TypeError: async_generate() missing 1 required positional argument: 'request_id'

## Mutated program + deepseek

In [ ]:
prompt = '''"""Fixtures for the lock entity platform tests."""

from collections.abc import Generator
from typing import Any
from unittest.mock import MagicMock

import pytest

from homeassistant.components.lock import (
    DOMAIN as LOCK_DOMAIN,
    LockEntity,
    LockEntityFeature,
)
from homeassistant.config_entries import ConfigEntry, ConfigFlow
from homeassistant.core import HomeAssistant
from homeassistant.helpers import entity_registry as er
from homeassistant.helpers.entity_platform import AddEntitiesCallback

from tests.common import (
    MockConfigEntry,
    MockModule,
    MockPlatform,
    mock_config_flow,
    mock_integration,
    mock_platform,
)
from typing import TypeAlias
__typ0 : TypeAlias = LockEntityFeature



TEST_DOMAIN = "test"


class MockLock(LockEntity):
    """Mocked lock entity."""

    def __init__(
        self,
        supported_features: __typ0 = __typ0(0),
        code_format: str | None = None,
    ) -> None:
        """Initialize the lock."""
        self.calls_open = MagicMock()
        self.calls_lock = MagicMock()
        self.calls_unlock = MagicMock()
        self._attr_code_format = code_format
        self._attr_supported_features = supported_features
        self._attr_has_entity_name = True
        self._attr_name = "test_lock"
        self._attr_unique_id = "very_unique_lock_id"
        super().__init__()

    def lock(self, **kwargs: <FILL>) -> None:
        """Mock lock lock calls."""
        self.calls_lock(**kwargs)

    def unlock(self, **kwargs: Any) -> None:
        """Mock lock unlock calls."""
        self.calls_unlock(**kwargs)

    def open(self, **kwargs: Any) -> None:
        """Mock lock open calls."""
        self.calls_open(**kwargs)


class MockFlow(ConfigFlow):
    """Test flow."""


@pytest.fixture(autouse=True)
def config_flow_fixture(hass: HomeAssistant) -> Generator[None, None, None]:
    """Mock config flow."""
    mock_platform(hass, f"{TEST_DOMAIN}.config_flow")

    with mock_config_flow(TEST_DOMAIN, MockFlow):
        yield


@pytest.fixture
async def code_format() -> str | None:
    """Return the code format for the test lock entity."""
    return None


@pytest.fixture(name="supported_features")
async def lock_supported_features() -> __typ0:
    """Return the supported features for the test lock entity."""
    return __typ0.OPEN


@pytest.fixture(name="mock_lock_entity")
async def setup_lock_platform_test_entity(
    hass: HomeAssistant,
    entity_registry: er.EntityRegistry,
    code_format: str | None,
    supported_features: __typ0,
) -> MagicMock:
    """Set up lock entity using an entity platform."""

    async def async_setup_entry_init(
        hass: HomeAssistant, config_entry: ConfigEntry
    ) -> bool:
        """Set up test config entry."""
        await hass.config_entries.async_forward_entry_setup(config_entry, LOCK_DOMAIN)
        return True

    MockPlatform(hass, f"{TEST_DOMAIN}.config_flow")
    mock_integration(
        hass,
        MockModule(
            TEST_DOMAIN,
            async_setup_entry=async_setup_entry_init,
        ),
    )

    # Unnamed sensor without device class -> no name
    entity = MockLock(
        supported_features=supported_features,
        code_format=code_format,
    )

    async def async_setup_entry_platform(
        hass: HomeAssistant,
        config_entry: ConfigEntry,
        async_add_entities: AddEntitiesCallback,
    ) -> None:
        """Set up test lock platform via config entry."""
        async_add_entities([entity])

    mock_platform(
        hass,
        f"{TEST_DOMAIN}.{LOCK_DOMAIN}",
        MockPlatform(async_setup_entry=async_setup_entry_platform),
    )

    config_entry = MockConfigEntry(domain=TEST_DOMAIN)
    config_entry.add_to_hass(hass)
    assert await hass.config_entries.async_setup(config_entry.entry_id)
    await hass.async_block_till_done()

    state = hass.states.get(entity.entity_id)
    assert state is not None

    return entity'''.strip()

original = '''
"""Fixtures for the lock entity platform tests."""

from collections.abc import Generator
from typing import Any
from unittest.mock import MagicMock

import pytest

from homeassistant.components.lock import (
    DOMAIN as LOCK_DOMAIN,
    LockEntity,
    LockEntityFeature,
)
from homeassistant.config_entries import ConfigEntry, ConfigFlow
from homeassistant.core import HomeAssistant
from homeassistant.helpers import entity_registry as er
from homeassistant.helpers.entity_platform import AddEntitiesCallback

from tests.common import (
    MockConfigEntry,
    MockModule,
    MockPlatform,
    mock_config_flow,
    mock_integration,
    mock_platform,
)

TEST_DOMAIN = "test"


class MockLock(LockEntity):
    """Mocked lock entity."""

    def __init__(
        self,
        supported_features: LockEntityFeature = LockEntityFeature(0),
        code_format: str | None = None,
    ) -> None:
        """Initialize the lock."""
        self.calls_open = MagicMock()
        self.calls_lock = MagicMock()
        self.calls_unlock = MagicMock()
        self._attr_code_format = code_format
        self._attr_supported_features = supported_features
        self._attr_has_entity_name = True
        self._attr_name = "test_lock"
        self._attr_unique_id = "very_unique_lock_id"
        super().__init__()

    def lock(self, **kwargs: <FILL>) -> None:
        """Mock lock lock calls."""
        self.calls_lock(**kwargs)

    def unlock(self, **kwargs: Any) -> None:
        """Mock lock unlock calls."""
        self.calls_unlock(**kwargs)

    def open(self, **kwargs: Any) -> None:
        """Mock lock open calls."""
        self.calls_open(**kwargs)


class MockFlow(ConfigFlow):
    """Test flow."""


@pytest.fixture(autouse=True)
def config_flow_fixture(hass: HomeAssistant) -> Generator[None, None, None]:
    """Mock config flow."""
    mock_platform(hass, f"{TEST_DOMAIN}.config_flow")

    with mock_config_flow(TEST_DOMAIN, MockFlow):
        yield


@pytest.fixture
async def code_format() -> str | None:
    """Return the code format for the test lock entity."""
    return None


@pytest.fixture(name="supported_features")
async def lock_supported_features() -> LockEntityFeature:
    """Return the supported features for the test lock entity."""
    return LockEntityFeature.OPEN


@pytest.fixture(name="mock_lock_entity")
async def setup_lock_platform_test_entity(
    hass: HomeAssistant,
    entity_registry: er.EntityRegistry,
    code_format: str | None,
    supported_features: LockEntityFeature,
) -> MagicMock:
    """Set up lock entity using an entity platform."""

    async def async_setup_entry_init(
        hass: HomeAssistant, config_entry: ConfigEntry
    ) -> bool:
        """Set up test config entry."""
        await hass.config_entries.async_forward_entry_setup(config_entry, LOCK_DOMAIN)
        return True

    MockPlatform(hass, f"{TEST_DOMAIN}.config_flow")
    mock_integration(
        hass,
        MockModule(
            TEST_DOMAIN,
            async_setup_entry=async_setup_entry_init,
        ),
    )

    # Unnamed sensor without device class -> no name
    entity = MockLock(
        supported_features=supported_features,
        code_format=code_format,
    )

    async def async_setup_entry_platform(
        hass: HomeAssistant,
        config_entry: ConfigEntry,
        async_add_entities: AddEntitiesCallback,
    ) -> None:
        """Set up test lock platform via config entry."""
        async_add_entities([entity])

    mock_platform(
        hass,
        f"{TEST_DOMAIN}.{LOCK_DOMAIN}",
        MockPlatform(async_setup_entry=async_setup_entry_platform),
    )

    config_entry = MockConfigEntry(domain=TEST_DOMAIN)
    config_entry.add_to_hass(hass)
    assert await hass.config_entries.async_setup(config_entry.entry_id)
    await hass.async_block_till_done()

    state = hass.states.get(entity.entity_id)
    assert state is not None

    return entity
'''.strip()
fim_type="Any"

In [ ]:
ds = load(f"{DIR}/experiments/deepseek_7b_base/nov8_rename_types.csv")

Generating train split: 0 examples [00:00, ? examples/s]/home/franlucc/venvs/vllm/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 45 examples [00:00, 3360.04 examples/s]


In [ ]:
# k = 0
# original = ds[k]["fim_program"]
# prompt = ds[k]["mutated_program"]
# fim_type = ds[k]["fim_type"]

original_answer = await async_generate(model, placeholder_to_std_fmt(original, DEEPSEEK_FIM),3)
new_answer = await async_generate(model, placeholder_to_std_fmt(prompt, DEEPSEEK_FIM),4)
original_answer, new_answer

INFO 11-08 15:41:39 async_llm_engine.py:174] Added request 3.
INFO 11-08 15:41:39 async_llm_engine.py:174] Added request 4.


(['Any'], ['Any'])

In [ ]:
new_answer

['Any']